In [23]:
import time
from datetime import datetime
import vk_api
import gspread
from oauth2client.service_account import ServiceAccountCredentials


def read_file(filename):
    try:
        with open(filename, 'r') as f:
            lines = [line.rstrip('\n') if line.endswith('\n') else line for line in f]
            if filename.endswith('groups.txt'):
                lines = ['-' + line for line in lines]
            return lines[0]
    except FileNotFoundError as e:
        print(f"Файл '{filename}' не найден.")
        exit(1)
    except Exception as e:
        print("Произошла ошибка при чтении файла: ", e)
        exit(1)


def get_or_create_worksheet(spreadsheet, worksheet_name):
    try:
        worksheet = spreadsheet.worksheet(worksheet_name)
    except gspread.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows="100", cols="20")
    return worksheet


def clear_worksheet(worksheet):
    worksheet.clear()
    worksheet.append_row(['Дата и время проверки', 'Дата публикации', 'Ссылка на пост', 'Текст', 'Комментарии', 'Лайки', 'Репосты',
                          'Охват подписчиков', 'Весь охват', 'Рекламный охват', 'Виральный охват',
                          'Переходов по ссылке', 'Переходов в сообщество',
                          'Вступлений', 'Жалоб',
                          'Скрыли запись', 'Отписались'])


def append_row_to_worksheet(worksheet, data):
    worksheet.append_row(data)
    time.sleep(1)  # Ограничение на 1 обращение к Google Таблицам в секунду


def main_loop():
    start_time = datetime.strptime('00:00:00', '%H:%M:%S').time()
    interval = 1 * 1 * 5

    token = read_file("../../access/tmar.txt")
    group_id = read_file("../../access/groups.txt")
    print(group_id)
    spreadsheet_key = read_file("../../../GS/Access/spreadsheet_id.txt")

    vk_session = vk_api.VkApi(token=token)
    api = vk_session.get_api()

    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('../../../GS/Access/credentials.json', scope)
    client = gspread.authorize(credentials)
    spreadsheet = client.open_by_key(spreadsheet_key)

    while True:
        current_time = datetime.now().time()
        if current_time >= start_time:
            worksheet_name = group_id
            worksheet = get_or_create_worksheet(spreadsheet, worksheet_name)
            clear_worksheet(worksheet)
            posts = api.wall.get(owner_id=group_id, count=200, extended=1)['items']
            if posts:
                for post in posts:
                    post_date = datetime.fromtimestamp(post['date']).strftime('%Y-%m-%d %H:%M:%S')
                    post_link = f"https://vk.com/wall-{group_id[1:]}_{post['id']}"
                    text = post['text']
                    comments = post.get('comments', {}).get('count', 0)
                    likes = post.get('likes', {}).get('count', 0)
                    reposts = post.get('reposts', {}).get('count', 0)
                    views = post.get('views', {}).get('count', 0)
                    reach_total = post.get('views', {}).get('reach_total', 0)
                    reach_ads = post.get('views', {}).get('reach_ads', 0)
                    reach_viral = post.get('views', {}).get('reach_viral', 0)
                    attachments = post.get('attachments', [])
                    link_clicks = 0
                    to_group = 0
                    join_group = 0
                    complaints = 0
                    hides = 0
                    unsubscribed = 0
                    for attachment in attachments:
                        if attachment['type'] == 'link':
                            link_clicks = attachment.get('link', {}).get('clicks', 0)
                            to_group = attachment.get('link', {}).get('to_group', 0)
                            join_group = attachment.get('link', {}).get('join_group', 0)
                            complaints = attachment.get('link', {}).get('complaints', 0)
                            hides = attachment.get('link', {}).get('hides', 0)
                            unsubscribed = attachment.get('link', {}).get('unsubscribed', 0)
                            break

                    data = [datetime.now().strftime('%Y-%m-%d %H:%M:%S'), post_date, post_link, text, comments, likes, reposts, views,
                            reach_total, reach_ads, reach_viral, link_clicks, to_group, join_group, complaints,
                            hides, unsubscribed]

                    append_row_to_worksheet(worksheet, data)

                    print(f'[{datetime.now()}] Записано: {post_date} - {post_link}')
            time.sleep(interval)
        else:
            time.sleep(60)


if __name__ == "__main__":
    main_loop()


-193643180
[2023-06-29 16:50:52.749333] Записано: 2023-06-29 13:08:39 - https://vk.com/wall-193643180_14
[2023-06-29 16:50:54.240278] Записано: 2023-06-29 10:33:22 - https://vk.com/wall-193643180_12
[2023-06-29 16:50:55.620896] Записано: 2020-06-30 19:12:04 - https://vk.com/wall-193643180_5


KeyboardInterrupt: 